In [4]:
# This use to generate explanability for SM VIT

In [6]:
# !pip3 install ml_collections

In [9]:
import torch

model = torch.load('models\SMVIT.pt',map_location=torch.device('cpu'))
# print(model.eval())

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has ch

In [24]:
import argparse
import cv2
import numpy as np
import torch
from vit_pytorch import ViT
import torch
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget


from pytorch_grad_cam import GradCAM, \
    ScoreCAM, \
    GradCAMPlusPlus, \
    AblationCAM, \
    XGradCAM, \
    EigenCAM, \
    EigenGradCAM, \
    LayerCAM, \
    FullGrad

from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image,preprocess_image
from pytorch_grad_cam.ablation_layer import AblationLayerVit


def reshape_transform(tensor, height=14, width=14):
    result = tensor[:, 1:, :].reshape(tensor.size(0),
                                      height, width, tensor.size(2))

    # Bring the channels to the first dimension,
    # like in CNNs.
    result = result.transpose(2, 3).transpose(1, 2)
    return result


methods = {"gradcam": GradCAM,
         "scorecam": ScoreCAM,
         "gradcam++": GradCAMPlusPlus }



# target_layers = [model.transformer.encoder.layer[0].attention_norm]
target_layers = [model.transformer.encoder.layer[1].attention_norm]
# target_layers = [model.transformer.encoder.layer[-1].attention_norm]

# method='gradcam'
method='gradcam'

cam = methods[method](model=model,target_layers=target_layers,use_cuda=False,reshape_transform=reshape_transform)

url='Sample Mel and Nev\Mel\ISIC_0029495.JPG'
rgb_img = cv2.imread(url, 1)[:, :, ::-1]
rgb_img = cv2.resize(rgb_img, (224, 224))
rgb_img = np.float32(rgb_img) / 255
input_tensor = preprocess_image(rgb_img, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

# If None, returns the map for the highest scoring category.
# Otherwise, targets the requested category.
targets = None
# input_tensor = input_tensor.cuda()
# AblationCAM and ScoreCAM have batched implementations.
# You can override the internal batch size for faster computation.
cam.batch_size = 32

skin_class=['Melanoma','Non-Melanoma']
outputs = cam.activations_and_grads(input_tensor)[0]
# print(outputs[0])

if targets is None:
    target_categories = np.argmax(outputs.cpu().data.numpy(), axis=-1)
    print(skin_class[target_categories[0]])
    targets = [ClassifierOutputTarget(category) for category in target_categories]

grayscale_cam = cam(input_tensor=input_tensor,aug_smooth =False,targets=targets)

# Here grayscale_cam has only one image in the batch
grayscale_cam = grayscale_cam[0, :]

cam_image = show_cam_on_image(rgb_img, grayscale_cam)

name=url.split('.')[0].split('/')[-1]
cv2.imwrite(f''+name+"_"+method+'.jpg', cam_image)

Melanoma


True